In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import random as rnd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble  import HistGradientBoostingClassifier
#from sklearn.ensemble  import HistGradientBoostingRegressor
from lightgbm import LGBMClassifier
import lightgbm as lgb
from  warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

In [ ]:
print('bulbar_vs_limb:\n')

In [ ]:
train = pd.read_csv("../input/end-als/end-als/transcriptomics-data/DESeq2/bulbar_vs_limb.csv")
train

****Sample number is only 112****

In [ ]:
train['SiteOnset_Class'].value_counts()

In [ ]:
X_train = train.drop(['SiteOnset_Class','Participant_ID'], axis=1)
y_train = train["SiteOnset_Class"]

In [ ]:
SVM = SVC()
RF = RandomForestClassifier()
LR = LogisticRegression()
HGBC = HistGradientBoostingClassifier()
LGB= LGBMClassifier()

scores = []
modelnames = ['HistGradient','SVM','RandumForest','LightGBM','LogisticRegression']
models = [HGBC,SVM,RF,LGB,LR]

In [ ]:
for i in models:
    score = cross_val_score(i, X_train, y_train, scoring = 'accuracy' , cv = 5).mean()
    scores.append(score)

pd.DataFrame(scores, index=modelnames,
            columns=['CV Scores']).sort_values(by = 'CV Scores', ascending = False)    

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE

#score_calc = 'neg_mean_absolute_error'
#score_calc = 'r2'
#score_calc = 'neg_mean_squared_log_error'
score_calc = 'accuracy'

In [ ]:
param_grid_lgb = { 'max_depth':[-1],
             # 'min_samples_leaf':[5],
             # 'num_leaves':[80],
             # 'min_child_samples' :[3],
             # 'learning_rate'  :[0.02],
             # 'max_bin':[265],
            # 'num_iterations':[300],
            #  "feature_fraction": [0.5], 
            #  "bagging_freq": 1,
           #  "bagging_fraction": [0.1] ,
              "seed":[42] ,
           #   'min_data_in_leaf': [50],
            # 'neg_bagging_fraction':[0.2],
            #  'reg_alpha': [0.05],
           #   'reg_lambda': [0.1],   
                 } 

In [ ]:
grid_searchlog_lgb = GridSearchCV(LGB, param_grid_lgb, cv =5, scoring = score_calc)

In [ ]:
grid_searchlog_lgb.fit(X_train, y_train)

In [ ]:
#LGB SCORE
#print(grid_searchlog_lgb.score(X_train, y_train))
print(grid_searchlog_lgb.best_params_)
print(grid_searchlog_lgb.best_score_)

In [ ]:
optimised_lgb = grid_searchlog_lgb.best_estimator_

****TOP30 Feature importance from LGBM ****

In [ ]:
lgb.plot_importance(optimised_lgb, figsize=(12, 8),max_num_features = 30,)
#plt.show()

****Use top 10 Feature****

In [ ]:
X_train2 = train[["CROCC2","NBPF3","MEFV","AC005597.1","AC015910.1","AC090617.7","AL137804.1","PKMP4","ENSG00000266701","CALB2"]]

In [ ]:
X_train2

In [ ]:
scores2 = []
for i in models:
    score2 = cross_val_score(i, X_train2, y_train, scoring = 'accuracy' , cv = 5).mean()
    scores2.append(score2)

pd.DataFrame(scores2, index=modelnames,
            columns=['CV Scores']).sort_values(by = 'CV Scores', ascending = False)    

In [ ]:
grid_searchlog_lgb.fit(X_train2, y_train)

In [ ]:
#LGB SCORE
#print(grid_searchlog_lgb.score(X_train, y_train))
print(grid_searchlog_lgb.best_params_)
print(grid_searchlog_lgb.best_score_)

In [ ]:
optimised_lgb = grid_searchlog_lgb.best_estimator_

In [ ]:
lgb.plot_importance(optimised_lgb, figsize=(12, 8),max_num_features = 10,)


****Check the 10 features histgram****

In [ ]:
import matplotlib.pyplot as plt
from pylab import rcParams

rcParams['figure.figsize'] = 10, 10 

X_train2.hist(); 

plt.tight_layout() 

plt.show() 

****Check corrmat****

In [ ]:
X_train2['SiteOnset_Class'] = train['SiteOnset_Class']

In [ ]:
corrmat = x = X_train2.corr()
k = 11
cols = corrmat.nlargest(k, 'SiteOnset_Class')['SiteOnset_Class'].index


In [ ]:
import seaborn as sns
cm = np.corrcoef(x = X_train2[cols].values.T)

sns.set(font_scale=1.25)

hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()
